**Tabla de contenido**

- [Requisitos técnicos](#Requisitos-tecnicos)
- [Definiendo la fábrica de modelos](#Definiendo-la-fabrica-de-modelos)
- [Diseñando tu sistema de entrenamiento](#Disenando-tu-sistema-de-entrenamiento)
    - [Opciones de diseño del sistema de entrenamiento](#Opciones-de-diseno-del-sistema-de-entrenamiento)
    - [Train-run](#Train-run)
    - [Train-persist](#Train-persist)
    - [Retraining required](#Retraining-required)
    - [Detectando el drift](#Detectando-el-drift)
- [Ingeniería de características para el consumo](#Ingenieria-de-caracteristicas-para-el-consumo)
    - [Engineering categorical features](#Engineering-categorical-features)
    - [Engineering numerical features](#Engineering-numerical-features)
    - [Definiendo el objetivo](#Definiendo-el-objetivo)
    - [Reducir tus pérdidas](#Reducir-tus-perdidas)
    - [Jerarquías de automatización](#Jerarquias-de-automatizacion)
- [Optimizing hyperparameters](#Optimizing-hyperparameters)
- [Auto-sklearn](#Auto-sklearn)
- [Optuna en redes neuronales](#Optuna-en-redes-neuronales)
- [Persisting your models](#Persisting-your-models)
- [Construyendo la fábrica de modelos con pipelines](#Construyendo-la-fabrica-de-modelos-con-pipelines)
    - [Scikit-learn pipelines](#Scikit-learn-pipelines)
    - [Spark ML pipelines](#Spark-ML-pipelines)

Este capítulo trata sobre uno de los conceptos más importantes en la ingeniería de ML: ¿cómo puedes tomar la difícil tarea de entrenar y ajustar tus modelos y convertirla en algo que puedas automatizar, reproducir y escalar para sistemas de producción?

Recapitularemos las ideas principales detrás del entrenamiento de diferentes modelos de ML a nivel teórico y práctico, antes de proporcionar motivación para el reentrenamiento, a saber, la idea de que los modelos de ML no rendirán bien para siempre. Este concepto también se conoce como drift. A continuación, cubriremos algunos de los conceptos principales detrás de la ingeniería de características, que es una parte clave de cualquier tarea de ML. A continuación, profundizaremos en cómo funciona el ML y cómo es, en esencia, una serie de problemas de optimización. Exploraremos cómo, al abordar estos problemas de optimización, puedes hacerlo con una variedad de herramientas en diferentes niveles de abstracción. En particular, discutiremos cómo puedes proporcionar la definición directa del modelo que deseas entrenar, a lo que llamo 'manualmente', o cómo puedes realizar la optimización de hiperparámetros o Aprendizaje Automático Automatizado (AutoML). Veremos ejemplos de uso de diferentes bibliotecas y herramientas que hacen todo esto, antes de explorar cómo implementarlas para su uso posterior en tu flujo de trabajo de entrenamiento.

A continuación, construiremos sobre el trabajo introductorio que hicimos en el Capítulo 2, El Proceso de Desarrollo de Aprendizaje Automático, sobre MLflow, mostrándote cómo interactuar con las diferentes API de MLflow para gestionar tus modelos y actualizar sus estados en el Registro de Modelos de MLflow.

Terminará este capítulo discutiendo las utilidades que le permiten encadenar todos sus pasos de entrenamiento de modelos de ML en unidades individuales conocidas como pipelines, que pueden ayudar a actuar como representaciones más compactas de todos los pasos que hemos discutido anteriormente. El resumen al final recapitulará los mensajes clave y también señalará cómo lo que hemos hecho aquí se desarrollará más en el Capítulo 4, Empaquetado, y el Capítulo 5, Patrones y Herramientas de Despliegue.

En esencia, este capítulo te dirá lo que necesitas para unir en tu solución, mientras que los capítulos posteriores te dirán cómo unirlos de manera robusta. Cubriremos esto en las siguientes secciones:

- Defining the model factory
- Designing your training system
- Retraining required
- Learning about learning
- Persisting your models
- Building the model factory with pipelines

# Requisitos tecnicos

Para completar este capítulo, necesitarás haber instalado los siguientes paquetes y herramientas de Python.

- MLflow: gestión del ciclo de vida de los modelos.
- TensorFlow
- auto-keras
- Hyperopt
- Optuna
- auto-sklearn
- alibi-detect: monitoreo de modelos en producción.

# Definiendo la fabrica de modelos

Si queremos desarrollar soluciones que se alejen de una ejecución ad hoc, manual e inconsistente y se dirijan hacia sistemas de aprendizaje automático que puedan ser automatizados, robustos y escalables, entonces debemos abordar la cuestión de cómo crearemos y curaremos la estrella del espectáculo: los propios modelos.

En este capítulo, discutiremos los componentes clave que deben ser reunidos para avanzar hacia esta visión y proporcionaremos algunos ejemplos de cómo pueden verse en código. Estos ejemplos no son la única forma de implementar estos conceptos, pero nos permitirán comenzar a construir nuestras soluciones de ML hacia el nivel de sofisticación que necesitaremos si queremos desplegar en el mundo real.

Los principales componentes de los que estamos hablando aquí son los siguientes:

- `Training system`: Un sistema para entrenar de manera robusta nuestros modelos con los datos que tenemos de forma automatizada. Esto consiste en todo el código que hemos desarrollado para entrenar nuestros modelos de aprendizaje automático con los datos.
- `Model Store`: Un lugar para persistir modelos entrenados con éxito y un lugar para compartir modelos listos para producción con componentes que ejecutarán las predicciones.

- `Drift detector`: un sistema para detectar cambios en el rendimiento del modelo que desencadenen ejecuciones de entrenamiento.

Estos componentes, combinados con su interacción con el sistema de predicción implementado, abarcan la idea de una fábrica de modelos.

En el resto de este capítulo, exploraremos en detalle los tres componentes que mencionamos anteriormente. Los sistemas de predicción serán el enfoque de capítulos posteriores, especialmente el Capítulo 5, Patrones y Herramientas de Despliegue. Primero, exploremos qué significa entrenar un modelo de ML y cómo podemos construir sistemas para hacerlo.

# Disenando tu sistema de entrenamiento

Visto en el nivel más alto, los modelos de ML pasan por un ciclo de vida con dos etapas: una fase de entrenamiento y una fase de salida. Durante la fase de entrenamiento, se alimenta al modelo con datos para aprender del conjunto de datos. En la fase de predicción, el modelo, completo con sus parámetros optimizados, recibe nuevos datos en orden y devuelve la salida deseada.

Estas dos fases tienen requisitos computacionales y de procesamiento muy diferentes. En la fase de entrenamiento, tenemos que exponer al modelo a la mayor cantidad de datos posible para obtener el mejor rendimiento, todo mientras aseguramos que se reserve un subconjunto de datos para pruebas y validación. El entrenamiento del modelo es fundamentalmente un problema de optimización, que requiere varios pasos incrementales para llegar a una solución. Por lo tanto, esto es computacionalmente exigente, y en casos donde los datos son relativamente grandes (o los recursos computacionales son relativamente bajos), puede tomar mucho tiempo.

Incluso si tuvieras un conjunto de datos pequeño y muchos recursos computacionales, el entrenamiento todavía no es un proceso de baja latencia. Además, es un proceso que a menudo se ejecuta en lotes y donde pequeñas adiciones al conjunto de datos no harán tanta diferencia en el rendimiento del modelo (hay excepciones a esto). La predicción, por otro lado, es un proceso más simple y se puede pensar de la misma manera que ejecutar cualquier cálculo o función en tu código: las entradas entran, se realiza un cálculo y el resultado sale. Esto (en general) no es computacionalmente exigente y tiene baja latencia.

Tomando esto en conjunto, significa que, en primer lugar, tiene sentido separar estos dos pasos (entrenamiento y predicción) tanto lógicamente como en el código. En segundo lugar, significa que tenemos que considerar los diferentes requisitos de ejecución para estas dos etapas y construyéndolo en nuestros diseños de soluciones. Finalmente, necesitamos tomar decisiones sobre nuestro régimen de entrenamiento, incluyendo si programamos el entrenamiento en lotes, usamos aprendizaje incremental, o si debemos activar el entrenamiento basado en criterios de rendimiento del modelo. Estas son las partes clave de tu sistema de entrenamiento.

## Opciones de diseno del sistema de entrenamiento

Antes de crear cualquier diseño detallado de nuestro sistema de entrenamiento, siempre se aplicarán algunas preguntas generales.

- ¿Hay infraestructura disponible que sea apropiada para el problema? 
- ¿Dónde están los datos y cómo los alimentaremos al algoritmo? 
- ¿Cómo estoy probando el rendimiento del modelo?

En términos de infraestructura, esto puede depender mucho del modelo y los datos que estés utilizando para el entrenamiento. Si vas a entrenar una regresión lineal con datos que tienen tres características y tu conjunto de datos contiene solo 10,000 registros tabulares, es probable que puedas ejecutar esto en hardware de escala de portátil sin pensarlo demasiado. No son muchos datos, y tu modelo no tiene muchos parámetros libres. Si estás entrenando en un conjunto de datos mucho más grande, como uno que contiene 100 millones de registros tabulares, entonces podrías beneficiarte de la paralelización a través de algo como un clúster de Spark. Si, sin embargo, estás entrenando una red neuronal convolucional profunda de 100 capas con 1,000 imágenes, entonces es probable que desees usar una GPU. Hay muchas opciones, pero la clave es elegir lo correcto para el trabajo.

Con respecto a la cuestión de cómo alimentamos datos al algoritmo, esto puede ser no trivial. ¿Vamos a ejecutar una consulta SQL contra una base de datos alojada de forma remota? Si es así, ¿cómo nos estamos conectando a ella? ¿La máquina en la que estamos ejecutando la consulta tiene suficiente RAM para almacenar los datos? Si no, ¿necesitamos considerar el uso de un algoritmo que pueda aprender de manera incremental? Para las pruebas de rendimiento algorítmico clásico, necesitamos emplear los trucos bien conocidos del comercio de ML y realizar divisiones de entrenamiento/prueba/validación en nuestros datos. También necesitamos decidir qué estrategias de validación cruzada queremos emplear. Luego necesitamos seleccionar nuestra métrica de rendimiento del modelo y calcularla adecuadamente. Sin embargo, como ingenieros de ML, también estaremos interesados en otras medidas de rendimiento, como el tiempo de entrenamiento, el uso eficiente de la memoria, la latencia y (me atrevo a decirlo) el costo. Necesitaremos entender cómo podemos medir y luego optimizar estos también.

Siempre y cuando tengamos en cuenta estas cosas a medida que avancemos, estaremos en una buena posición. Ahora, pasemos al diseño. Como mencionamos en la introducción a esta sección, tenemos dos piezas fundamentales a considerar: los procesos de entrenamiento y output. Hay dos formas en las que podemos juntar esto para nuestra solución. Discutiremos esto en la próxima sección.

## Train-run

La opción 1 consiste en realizar el entrenamiento y la predicción en el mismo proceso, con el entrenamiento ocurriendo en modo por lotes o incremental. Este patrón se llama entrenar-ejecutar.

Este patrón es el más simple de los dos, pero también el menos deseable para problemas del mundo real, ya que no encarna el principio de separación de preocupaciones que mencionamos anteriormente. Esto no significa que sea un patrón inválido, y tiene la ventaja de ser a menudo más simple de implementar. Aquí, ejecutamos todo nuestro proceso de entrenamiento antes de hacer nuestras predicciones, sin una pausa real entre ellos. Dadas nuestras discusiones anteriores, podemos descartar automáticamente este enfoque si tenemos que servir predicciones de manera muy baja en latencia; por ejemplo, a través de una solución basada en eventos o por streaming (más sobre esto más adelante).

Donde este enfoque podría ser completamente válido, aunque (y he visto esto algunas veces en la práctica), es en casos donde los algoritmos que estás aplicando son realmente muy ligeros de entrenar y necesitas seguir usando datos muy recientes, o donde estás ejecutando un proceso por lotes grande relativamente infrecuentemente.

Aunque este es un enfoque simple y no se aplica a todos los casos, tiene ventajas distintas:

- Dado que te estás entrenando tan a menudo como predices, estás haciendo todo lo posible para protegerte contra la degradación del rendimiento moderno, lo que significa que estás combatiendo el driff (deslizamiento).

- Estás reduciendo significativamente la complejidad de tu solución. Aunque estás acoplando estrechamente dos componentes, lo cual generalmente debería evitarse, las etapas de entrenamiento y predicción pueden ser tan simples de codificar que si simplemente las unes, ahorrarás mucho tiempo de desarrollo. Este es un punto no trivial porque el tiempo de desarrollo cuesta dinero.

Ahora, veamos el otro caso.

## Train-persist

La opción 2 es que el entrenamiento se realice en lote, mientras que la predicción funcione en el modo que se considere apropiado, con la solución de predicción leyendo el modelo entrenado de un almacenamiento. Llamaremos a este patrón de diseño entrenar-persistir.

Si vamos a entrenar nuestro modelo y luego a persistir el modelo para que pueda ser recogido más tarde por un proceso de predicción, entonces necesitamos asegurarnos de que algunas cosas estén en su lugar:

- ¿Cuáles son nuestras opciones de almacenamiento de modelos?
- ¿Hay un mecanismo claro para acceder a nuestro almacén de modelos (escribir y leer)?
- ¿Con qué frecuencia deberíamos entrenar frente a con qué frecuencia predeciremos?

En nuestro caso, resolveremos las dos primeras preguntas utilizando MLflow, que introdujimos en el Capítulo 2, El Proceso de Desarrollo de Aprendizaje Automático, pero que revisitaremos en secciones posteriores. También hay muchas otras soluciones disponibles. El punto clave es que, sin importar qué utilices como almacén de modelos y punto de entrega entre tus procesos de entrenamiento y predicción, debe ser utilizado de manera que sea robusto y accesible.

El tercer punto es más complicado. Podrías decidir desde el principio que quieres entrenar en un horario y apegarte a eso. O podrías ser más sofisticado y desarrollar criterios de activación que deben cumplirse antes de que ocurra el entrenamiento. Nuevamente, esta es una elección que tú, como ingeniero de ML, necesitas hacer con tu equipo. Más adelante en este capítulo, discutiremos mecanismos para programar tus sesiones de entrenamiento.

En la siguiente sección, exploraremos qué debes hacer si deseas activar tus ejecuciones de entrenamiento basándote en cómo el rendimiento de tu modelo podría estar degradándose con el tiempo.


## Retraining required

No esperarías que después de terminar tu educación, nunca más leas un artículo o un libro o hables con nadie, lo que significa que no podrías tomar decisiones informadas sobre lo que está sucediendo en el mundo. Así que no deberías esperar que un modelo de aprendizaje automático sea entrenado una vez y luego funcione bien para siempre.

Esta idea es intuitiva, pero representa un problema formal para los modelos de ML conocido como drift. Drift es un término que abarca una variedad de razones por las cuales el rendimiento de su modelo disminuye con el tiempo. Se puede dividir en dos tipos principales:

- `Concept drift`: Esto ocurre cuando hay un cambio en la relación fundamental entre las características de tus datos y el resultado que estás tratando de predecir. A veces, esto también se conoce como desviación de covariables. Un ejemplo podría ser que en el momento del entrenamiento, solo tienes una submuestra de datos que parece mostrar una relación lineal entre las características y tu resultado. Si resulta que, después de recopilar muchos más datos tras el despliegue, la relación es no lineal, entonces ha ocurrido una desviación conceptual. La mitigación contra esto es el reentrenamiento con datos que sean más representativos de la relación correcta.
- `Data drift`: Esto sucede cuando hay un cambio en las propiedades estadísticas de las variables que estás utilizando como características. Por ejemplo, podrías estar usando la edad como una característica en uno de tus modelos, pero en el momento del entrenamiento, solo tienes datos de personas de 16 a 24 años. Si el modelo se despliega y tu sistema comienza a ingerir datos de una demografía de edad más amplia, entonces tienes desviación de datos.

Detectar el desvío en tus modelos desplegados es una parte clave de MLOps y debe ser una prioridad en la mente de un ingeniero de ML. Si puedes construir tus sistemas de entrenamiento de tal manera que el reentrenamiento se active en función de una comprensión informada del desvío en tus modelos, ahorrarás muchos recursos computacionales al entrenar solo cuando sea necesario.

La siguiente sección discutirá algunas de las maneras en que podemos detectar el cambio en nuestros modelos. Esto nos ayudará a comenzar a construir una estrategia de reentrenamiento inteligente en nuestra solución.

## Detectando el drift

Hasta ahora, hemos definido el drift y sabemos que detectarlo será importante si queremos construir sistemas de entrenamiento sofisticados. La siguiente pregunta lógica es: ¿cómo hacemos esto? Las definiciones de drift que dimos en la sección anterior eran muy cualitativas; podemos comenzar a hacer estas afirmaciones un poco más cuantitativas a medida que exploramos los cálculos y conceptos que pueden ayudarnos a detectar el drift.

En esta sección, nos basaremos en gran medida en el paquete de Python alibi-detect de Seldon, que, al momento de escribir, no estaba disponible en Anaconda.org pero está disponible en PyPI. Para adquirir este paquete, utiliza los siguientes comandos:

- pip install tensorflow[and-cuda]
- pip install tensorflow-probability
- pip install tf-keras
- pip install alibi
- pip install alibi-detect

Es muy fácil de usar el paquete alibi-detect. En el siguiente ejemplo, trabajaremos con el conjunto de datos de vino de sklearn, que se utilizará en otras partes de este capítulo. En este primer ejemplo, dividiremos los datos 50/50 y llamaremos a un conjunto el conjunto de referencia y al otro el conjunto de prueba. Luego utilizaremos la prueba de Kolmogorov-Smirnov para demostrar que no ha habido drift de datos entre estos dos conjuntos de datos, como se esperaba, y luego agregaremos artificialmente algo de drift para mostrar que ha sido detectada con éxito:

In [ ]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
import alibi
from alibi_detect.cd import TabularDrift

A continuación, debemos obtener y dividir los datos:

In [ ]:
wine_data = load_wine()
feature_names = wine_data.feature_names
X, y = wine_data.data, wine_data.target
X_ref, X_test, y_ref, y_test = train_test_split(X, y,test_size=0.50,random_state=42)

A continuación, debemos inicializar nuestro detector de drift utilizando los datos de referencia y proporcionando el valor p que queremos que se utilice en las pruebas de significancia estadística. Si deseas que tu detector de drift se active cuando ocurran diferencias más pequeñas en la distribución de los datos, debes seleccionar un valor p más grande.

Cuando usas TabularDrift, el detector necesita saber si cada columna de tu dataset es numérica o categórica, porque:

- Para numéricas → aplica Kolmogorov–Smirnov test (KS test).

- Para categóricas → aplica Chi-cuadrado test u otras pruebas adecuadas.

Como no le das un diccionario categories_per_feature, Alibi-Detect asume que todas las columnas son numéricas y aplica KS test en todas.

In [ ]:
# Todas las columnas son numéricas
categories_per_feature = {i: None for i in range(X_ref.shape[1])}
cd = TabularDrift(x_ref=X_ref, p_val=.05,categories_per_feature=categories_per_feature)

Ahora podemos comprobar si hay drift (desvio) en el conjunto de datos de prueba en comparación con el conjunto de datos de referencia:

In [ ]:
preds = cd.predict(X_test)
labels = ['No', 'Yes']
print('Drift: {}'.format(labels[preds['data']['is_drift']]))

`Así que no hemos detectado deriva aquí, como se esperaba.`

Aunque no hubo drift en este caso, podemos simular fácilmente un escenario en el que el aparato químico utilizado para medir las propiedades químicas experimentó un error de calibración, y todos los valores se registran como un 10% más altos que sus valores reales. En este caso, si volvemos a ejecutar la detección de drift en el mismo conjunto de datos de referencia, obtendremos la siguiente salida:

In [ ]:
X_test_cal_error = 1.1*X_test
preds = cd.predict(X_test_cal_error)
labels = ['No', 'Yes']
print('Drift: {}'.format(labels[preds['data']['is_drift']]))

`NOTA IMPORTANTE`: Este ejemplo es muy artificial, pero es útil para ilustrar el punto. En un conjunto de datos estándar como este, no habrá deriva de datos entre el 50% de los datos muestreados aleatoriamente y el otro 50% de los datos. Por eso, tenemos que desplazar artificialmente algunos de los puntos para mostrar que el detector realmente funciona. En escenarios del mundo real, la deriva de datos puede ocurrir de forma natural debido a todo, desde actualizaciones de sensores utilizados para mediciones; hasta cambios en el comportamiento del consumidor; pasando por cambios en el software o esquemas de bases de datos. Así que, esté atento, ya que muchos casos de deriva no serán tan fáciles de detectar como en este caso!.

Este ejemplo muestra cómo, con unas pocas líneas simples de Python, podemos detectar un cambio en nuestro conjunto de datos, lo que significa que nuestro modelo de ML puede comenzar a degradarse en rendimiento si no lo reentrenamos para tener en cuenta las nuevas propiedades de los datos. También podemos utilizar técnicas similares para rastrear cuándo las métricas de rendimiento de nuestro modelo, por ejemplo, la precisión o el error cuadrático medio, también están desviándose. En este caso, debemos asegurarnos de calcular periódicamente el rendimiento en nuevos conjuntos de datos de prueba o validación. Ahora, podemos comenzar a integrar esto en soluciones que dispararán automáticamente el reentrenamiento de nuestro modelo de ML, como se muestra en el siguiente diagrama:

![Un ejemplo de detección de deriva y el proceso del sistema de entrenamiento](figures/Deteccion-de-deriva.png)

A continuación, veremos cómo diseñar características específicas para el consumo de datos.

# Ingenieria de caracteristicas para el consumo

Antes de alimentar cualquier dato a un modelo de aprendizaje automático, tiene que ser transformado en un estado que pueda ser entendido por nuestros modelos. También necesitamos asegurarnos de que solo hagamos esto con los datos que consideramos útiles para mejorar el rendimiento del modelo, ya que es demasiado fácil aumentar el número de características y caer en la maldición de la dimensionalidad. Esto se refiere a una serie de observaciones relacionadas donde, en problemas de alta dimensión, los datos se vuelven cada vez más dispersos en el espacio de características, por lo que lograr significancia estadística puede requerir exponencialmente más datos. En esta sección, no cubriremos la base teórica de la ingeniería de características. En cambio, nos enfocaremos en cómo nosotros, como ingenieros de ML, podemos ayudar a automatizar algunos de los pasos en producción. Para este fin, repasaremos rápidamente los principales tipos de preparación de características y pasos de ingeniería de características para que tengamos las piezas necesarias para agregar a nuestros flujos de trabajo más adelante en este capítulo.

## Engineering categorical features

Las características categóricas son aquellas que forman un conjunto no numérico de objetos distintos, como el día de la semana o el color de cabello. Pueden distribuirse de varias maneras a lo largo de sus datos. Para que un algoritmo de ML pueda digerir una característica categórica, necesitamos traducir la característica en algo numérico, mientras nos aseguramos de que la representación numérica no produzca sesgo o pese nuestros valores de manera inapropiada. Un ejemplo de esto sería si tuviéramos una característica que contenga diferentes productos vendidos en un supermercado:

In [ ]:
data = [['Bleach'], ['Cereal'], ['Toilet Roll']]

Aquí, podemos mapear cada uno a un entero positivo utilizando el OrdinalEncoder de sklearn.

In [ ]:
from sklearn import preprocessing
ordinal_enc = preprocessing.OrdinalEncoder()
ordinal_enc.fit(data)
print(ordinal_enc.transform(data))

Esto es lo que se llama codificación ordinal. Hemos mapeado estas características a números, así que ahí hay un gran avance, pero ¿es la representación adecuada? Bueno, si lo piensas por un segundo, no realmente. Estos números parecen sugerir que el cereal es a la lejía como el papel higiénico es al cereal, y que el promedio de papel higiénico y lejía es cereal. Estas afirmaciones no tienen sentido (y no quiero lejía y papel higiénico para el desayuno), así que esto sugiere que deberíamos intentar un enfoque diferente.

Esta representación sería apropiada, sin embargo, en casos donde quisiéramos mantener la noción de orden en las características categóricas. Un excelente ejemplo sería si tuviéramos una encuesta, y se les pidiera a los participantes su opinión sobre la afirmación de que el desayuno es la comida más importante del día. Si luego se les pidiera a los participantes que seleccionaran una opción de la lista Fuertemente en desacuerdo, En desacuerdo, Ni de acuerdo ni en desacuerdo, De acuerdo y Fuertemente de acuerdo, y codificáramos ordinalmente estos datos para mapear a la lista numérica de 1, 2, 3, 4 y 5, entonces podríamos responder más intuitivamente a preguntas como si la respuesta promedio estaba más de acuerdo o en desacuerdo? y cuán extendida estaba la opinión sobre esta afirmación?. La codificación ordinal ayudaría aquí, pero como mencionamos anteriormente, no es necesariamente correcta en este caso.

Lo que podríamos hacer es considerar la lista de elementos en esta característica y luego proporcionar un número binario para representar si el valor es o no ese valor particular en la lista original. Así que aquí, decidimos usar el OneHotEncoder de sklearn:

In [ ]:
onehot_enc = preprocessing.OneHotEncoder()
onehot_enc.fit(data)
print(onehot_enc.transform(data).toarray())

Esta representación se conoce como codificación one-hot. Hay algunos beneficios en este método de codificación, incluidos los siguientes:

- No hay ordenamientos impuestos de los valores. 
- Todos los vectores característicos tienen normas unitarias. 
- Cada característica única es ortogonal a las demás, por lo que no hay promedios o afirmaciones de distancia extrañas que estén implícitas en la representación.

Una de las desventajas de este enfoque es que, si tu lista categórica contiene muchas instancias, el tamaño de tu vector de características puede aumentar rápidamente, y tenemos que almacenar y trabajar con vectores y matrices extremadamente dispersos a nivel algorítmico. Esto puede llevar fácilmente a problemas en varias implementaciones y es otra manifestación de la temida maldición de la dimensionalidad.

## Engineering numerical features

Preparar características numéricas es un poco más fácil ya que ya tenemos números, pero hay algunos pasos que aún debemos seguir para prepararnos para muchos algoritmos. Para la mayoría de los algoritmos de ML, las características deben estar en escalas similares; por ejemplo, deben tener una magnitud entre -1 y 1 o entre 0 y 1. Esto es por la razón relativamente obvia de que algunos algoritmos que toman una característica para valores de precios de casas de hasta un millón de dólares y otra para la superficie en pies cuadrados de la casa automáticamente ponderarán más los valores en dólares más grandes.

Esto también significa que perdemos la noción útil de dónde se ubican los valores específicos en sus distribuciones. Por ejemplo, algunos algoritmos se beneficiarán de escalar las características para que el valor medio en dólares y el valor medio en pies cuadrados estén representados ambos por 0.5 en lugar de 500,000 y 350. O puede que queramos que todas nuestras distribuciones tengan el mismo significado si estuvieran distribuidas normalmente, lo que permite que nuestros algoritmos se concentren en la forma de las distribuciones en lugar de sus ubicaciones.

Entonces, ¿qué hacemos? Bueno, como siempre, no estamos empezando desde cero y hay algunas técnicas estándar que podemos aplicar. Algunas de las más comunes se enumeran aquí, pero hay muchas más para incluir todas ellas:

- `Feature vector normalization`: Aquí, escalarás cada muestra en tu conjunto de datos para que tengan normas iguales a 1. Esto puede ser muy importante si estás utilizando algoritmos donde la distancia o la similitud coseno entre características es un componente importante, como en el agrupamiento. También se utiliza comúnmente en la clasificación de texto en combinación con otros métodos de ingeniería de características, como la estadística TF-IDF. En este caso, asumiendo que toda tu característica es numérica, solo necesitas calcular la norma apropiada para tu vector de características y luego dividir cada componente por ese valor.

Primero, debemos importar las bibliotecas relevantes y configurar nuestros datos de entrenamiento y prueba:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifier
from sklearn import metrics
from sklearn.datasets import load_wine
from sklearn.pipeline import make_pipeline
X, y = load_wine(return_X_y=True)

Entonces, debemos hacer una división típica de 70/30 entre entrenamiento y prueba.

In [ ]:
X_train, X_test, y_train, y_test =\
train_test_split(X, y, test_size=0.30, random_state=42)

A continuación, debemos entrenar un modelo sin ninguna estandarización en las características y predecir en el conjunto de prueba.

In [ ]:
no_scale_clf = make_pipeline(RidgeClassifier(tol=1e-2, solver="sag"))
no_scale_clf.fit(X_train, y_train)
y_pred_no_scale = no_scale_clf.predict(X_test)

Finalmente, debemos hacer lo mismo pero con un paso de estandarización añadido.

In [ ]:
std_scale_clf = make_pipeline(StandardScaler(), RidgeClassifier(tol=1e-2,
solver="sag"))
std_scale_clf.fit(X_train, y_train)
y_pred_std_scale = std_scale_clf.predict(X_test)

Ahora, si imprimimos algunas métricas de rendimiento, veremos que sin escalado, la exactitud de las predicciones es de 0.76, mientras que las otras métricas, como los promedios ponderados de precisión, recuperación y f1-score, son 0.83, 0.76 y 0.68, respectivamente.

In [ ]:
print('\nAccuracy [no scaling]')
print('{:.2%}\n'.format(metrics.accuracy_score(y_test, y_pred_no_scale)))
print('\nClassification Report [no scaling]')
print(metrics.classification_report(y_test, y_pred_no_scale))

En el caso en que estandarizamos los datos, las métricas son mucho mejores en general, con la precisión y los promedios ponderados de la precisión, recuperación y f1-score todos en 0.98.

In [ ]:
print('\nAccuracy [scaling]')
print('{:.2%}\n'.format(metrics.accuracy_score(y_test, y_pred_std_scale)))
print('\nClassification Report [scaling]')
print(metrics.classification_report(y_test, y_pred_std_scale))

Aquí, podemos ver un salto significativo en el rendimiento, simplemente añadiendo un paso simple a nuestro proceso de entrenamiento de ML. Ahora, veamos cómo funciona el entrenamiento en su esencia. Esto nos ayudará a tomar decisiones sensatas para nuestros algoritmos y enfoques de entrenamiento.

# Aprendiendo sobre el aprendizaje

En su esencia, los algoritmos de ML contienen una característica clave: una optimización de algún tipo. El hecho de que estos algoritmos aprendan (lo que significa que mejoran iterativamente su rendimiento en relación con una métrica apropiada al exponerse a más observaciones) es lo que los hace tan poderosos y emocionantes. Este proceso de aprendizaje es lo que nos referimos cuando decimos entrenamiento.

En esta sección, cubriremos los conceptos clave que sustentan el entrenamiento, las opciones que podemos seleccionar en nuestro código y lo que esto significa para el rendimiento potencial y las capacidades de nuestro sistema de entrenamiento.


## Definiendo el objetivo

Acabamos de afirmar que el entrenamiento es una optimización, pero ¿qué estamos optimizando exactamente? Consideremos el aprendizaje supervisado. En el entrenamiento, proporcionamos las etiquetas o valores que deseamos predecir para la característica dada, de modo que los algoritmos puedan aprender la relación entre las características y el objetivo. Para optimizar los parámetros internos del algoritmo durante el entrenamiento, necesita saber cuán incorrecto estaría con su conjunto actual de parámetros. La optimización, entonces, se trata de actualizar los parámetros para que esta medida de incorrectitud sea cada vez más pequeña. Esto es exactamente lo que se capta con el concepto de una función de pérdida.

Las funciones de pérdida vienen en una variedad de formas, y puedes incluso definir la tuya propia si lo necesitas con muchos paquetes, pero hay algunas estándar de las que es útil estar al tanto. Los nombres de algunas de estas se mencionan aquí.

Para problemas de regresión, puedes usar lo siguiente:

- Mean squared error/L2 loss
- Mean absolute error/L1 loss

Para problemas de clasificación binaria, puedes usar lo siguiente:

- Log loss/logistic loss/cross-entropy loss
- Hinge loss

Para problemas de clasificación multiclase, puedes utilizar lo siguiente:

- Multi-class across entropy loss
- Kullback Leibler Divergence loss

Después de definir tu función de pérdida, luego necesitas optimizarla. Esto es lo que analizaremos en la próxima sección.



## Reducir tus perdidas

En este punto, sabemos que el entrenamiento se trata de optimización, y sabemos qué optimizar, pero aún no hemos cubierto cómo optimizar. Como de costumbre, hay muchas opciones para elegir. En esta sección, examinaremos algunos de los enfoques principales.

Las siguientes son las enfoques de tasa de aprendizaje constante:

- `Gradient descent`: Este algoritmo funciona calculando la derivada de nuestra función de pérdida respecto a nuestros parámetros, y luego utiliza esto para construir una actualización que nos mueve en la dirección de disminuir la pérdida.

- `Batch gradient descent`: El gradiente que usamos para hacer nuestro movimiento en el espacio de parámetros se encuentra tomando el promedio de todos los gradientes encontrados. Esto lo hace observando cada punto de datos en nuestro conjunto de entrenamiento y comprobando si el conjunto de datos no es demasiado grande y si la función de pérdida es relativamente suave y convexa. Esto puede alcanzar prácticamente el mínimo global.

- `Stochastic gradient descent`: El gradiente se calcula utilizando un punto de datos seleccionado al azar en cada iteración. Esto es más rápido para llegar al mínimo global de la función de pérdida, pero es más susceptible a fluctuaciones repentinas en la pérdida después de cada paso de optimización.

- `Mini-batch gradient descent`: Esta es una mezcla de los casos por lotes y estocásticos. En este caso, las actualizaciones del gradiente para cada actualización de los parámetros utilizan varios puntos mayores a uno pero menores que todo el conjunto de datos. Esto significa que el tamaño del lote es ahora un parámetro que necesita ser ajustado. Cuanto mayor es el lote, más nos acercamos al descenso de gradiente por lotes, que proporciona una mejor estimación del gradiente pero es más lento. Cuanto menor es el lote, más nos acercamos al descenso de gradiente estocástico, que es más rápido pero no tan robusto. El mini-lote nos permite decidir dónde queremos estar entre los dos. Los tamaños de lote pueden seleccionarse con una variedad de criterios en mente. Estos pueden tener en cuenta una serie de consideraciones de memoria. Los lotes procesados en paralelo y los lotes más grandes consumirán más memoria mientras proporcionan un mejor rendimiento de generalización para lotes más pequeños. Consulte el Capítulo 8 del libro Deep Learning de Ian Goodfellow, Yoshua Bengio y Aaron Courville en https://www.deeplearningbook.org/ para más detalles.

Luego, están los métodos de tasa de aprendizaje adaptativa. Algunos de los más comunes son los siguientes:

- `AdaGrad`: Los parámetros de la tasa de aprendizaje se actualizan dinámicamente en función de las propiedades de las actualizaciones de aprendizaje durante el proceso de optimización.
- `AdaDelta`: Esta es una extensión de AdaGrad que no utiliza todas las actualizaciones de gradiente anteriores. En cambio, utiliza una ventana deslizante sobre las actualizaciones.
- `RMSprop`: Esto funciona manteniendo un promedio móvil del cuadrado de todos los pasos del gradiente. Luego, divide el último gradiente por la raíz cuadrada de esto.
- `Adam`: Este es un algoritmo que se supone que combina los beneficios de AdaGrad y RMSprop.

Los límites y capacidades de todos estos enfoques de optimización son importantes para nosotros, como ingenieros de aprendizaje automático, porque queremos asegurar que nuestros sistemas de entrenamiento utilicen la herramienta adecuada para el trabajo y sean óptimos para el problema en cuestión. Simplemente tener la conciencia de que hay múltiples opciones para su optimización interna también te ayudará a enfocar tus esfuerzos y aumentar el rendimiento.

Ahora, pensemos en qué nivel de control podemos tener sobre el proceso de entrenamiento mientras desarrollamos nuestras soluciones.

## Jerarquias de automatizacion

Una de las principales razones por las que el aprendizaje automático (ML) es ahora una parte común del desarrollo de software, así como una actividad empresarial y académica importante, es debido a la plétora de herramientas disponibles en la actualidad. Todos los paquetes y bibliotecas que contienen implementaciones funcionales y optimizadas de algoritmos sofisticados han permitido a las personas construir sobre ellos, en lugar de tener que reimplementar lo básico cada vez que hay un problema por resolver. Esta es una poderosa expresión de la idea de abstracción en el desarrollo de software, donde se pueden aprovechar y utilizar unidades de nivel inferior en niveles más altos de implementación.

Esta idea se puede ampliar aún más a toda la empresa de la capacitación en sí. En el nivel más bajo de implementación (pero aún muy alto en el sentido de los algoritmos subyacentes), podemos proporcionar detalles sobre cómo queremos que avance el proceso de capacitación. Podemos definir manualmente el conjunto exacto de hiperparámetros (ver la siguiente sección sobre Optimización de hiperparámetros) que usar en la ejecución de la capacitación en nuestro código. Yo llamo a esto 'manipular a mano'. Luego podemos pasar un nivel de abstracción superior y proporcionar rangos y límites para nuestros hiperparámetros a herramientas diseñadas para muestrear y probar de manera eficiente el rendimiento de nuestro modelo para cada uno de estos; por ejemplo, la sintonización automática de hiperparámetros. Finalmente, hay un nivel de abstracción aún más alto que ha creado mucha emoción mediática en los últimos años, donde optimizamos sobre qué algoritmo ejecutar. Esto es conocido como ML automatizado o AutoML.

Hay mucho bombo alrededor de AutoML, con algunas personas proclamando la eventual automatización de todos los roles laborales en el desarrollo de ML. En mi opinión, esto simplemente no es realista, ya que seleccionar tu modelo y los hiperparámetros es solo un aspecto de un desafío de ingeniería enormemente complejo (de ahí que esto sea un libro y no un folleto). Sin embargo, AutoML es una herramienta muy poderosa que debería añadirse a tu arsenal de capacidades cuando te enfrentes a tu próximo proyecto de ML.

Podemos resumir todo esto de manera bastante sencilla como una jerarquía de automatización; básicamente, ¿cuánto control quieres tú, como ingeniero de ML, en el proceso de entrenamiento? Una vez escuché que esto se describía en términos de control de marchas en un coche (crédito: Databricks en Spark AI 2019). Manejar a mano es el equivalente de conducir un coche manual, con control total sobre las marchas: hay más en qué pensar, pero puede ser muy eficiente si sabes lo que estás haciendo. Un nivel más arriba, tienes coches automáticos: hay menos de qué preocuparse, para que puedas concentrarte más en llegar a tu destino, el tráfico y otros desafíos.

Esta es una buena opción para muchas personas, pero aún requiere que tengas suficientes conocimientos, habilidades y comprensión. Finalmente, tenemos coches autónomos: siéntate, relájate y ni siquiera te preocupes por cómo llegar a donde vas. Puedes centrarte en lo que vas a hacer una vez que llegues allí.

# Optimizing hyperparameters

Una de las principales razones por las que el aprendizaje automático (ML) es ahora una parte común del desarrollo de software, así como una actividad empresarial y académica importante, es debido a la plétora de herramientas disponibles en la actualidad. Todos los paquetes y bibliotecas que contienen implementaciones funcionales y optimizadas de algoritmos sofisticados han permitido a las personas construir sobre ellos, en lugar de tener que reimplementar lo básico cada vez que hay un problema por resolver. Esta es una poderosa expresión de la idea de abstracción en el desarrollo de software, donde se pueden aprovechar y utilizar unidades de nivel inferior en niveles más altos de implementación.

Cuando ajustas algún tipo de función matemática a los datos, algunos valores se ajustan durante el procedimiento de ajuste o entrenamiento: estos se llaman parámetros. Para el aprendizaje automático, hay un nivel adicional de abstracción donde tenemos que definir los valores que informan a los algoritmos que estamos empleando cómo deben actualizar los parámetros. Estos valores se llaman hiperparámetros, y su selección es una de las importantes artes ocultas del entrenamiento de algoritmos de aprendizaje automático.

Las siguientes tablas enumeran algunos hiperparámetros que se utilizan para algoritmos comunes de ML para mostrarte las diferentes formas que pueden tomar. Estas listas no son exhaustivas, pero están ahí para resaltar que la optimización de hiperparámetros no es un ejercicio trivial:

| Algorithm                   | Hyperparameters                               | What This Controls                          |
|------------------------------|-----------------------------------------------|---------------------------------------------|
| Decision Trees and Random Forests | Tree depth<br> Min/max leaves | Number of levels<br> Amount of branching at each level |
| Support Vector Machines (SVM) | C<br> Gamma | Penalty for misclassification<br> Radius of influence (RBF kernel) |
| Neural Networks              | Learning rate<br> Number of hidden layers<br> Activation function | Update step size<br> Depth of the network<br> Neuron activation conditions |
| Logistic Regression          | Solver<br> Regularization type<br> Regularization strength | How to minimize the loss<br> How to prevent overfitting<br> Strength of regularization |


Todos estos hiperparámetros tienen su propio conjunto específico de valores que pueden tomar. Este rango de valores de hiperparámetros para los diferentes algoritmos potenciales que deseas aplicar a tu solución de ML significa que hay muchas maneras de definir un modelo funcional (es decir, uno que no rompa la implementación que estás utilizando), pero ¿cómo encuentras el modelo óptimo?

Aquí es donde entra la búsqueda de hiperparámetros. El concepto es que, para un número finito de combinaciones de valores de hiperparámetros, queremos encontrar el conjunto que proporcione el mejor rendimiento del modelo. ¡Este es otro problema de optimización que es similar al de la capacitación en primer lugar!

En las siguientes secciones, discutiremos dos bibliotecas de optimización de hiperparámetros muy populares y te mostraremos cómo implementarlas en unas pocas líneas de Python.

`NOTA IMPORTANTE`: Es importante entender qué algoritmos se están utilizando para la optimización en estas bibliotecas de hiperparámetros, ya que puedes querer usar un par de implementaciones diferentes de cada una para comparar diferentes enfoques y evaluar el rendimiento. Si no observaste cómo funcionan internamente, podrías hacer comparaciones injustas con facilidad, o peor, podrías estar comparando casi lo mismo sin saberlo. Si tienes un conocimiento más profundo de cómo funcionan estas soluciones, también podrás tomar mejores decisiones sobre cuándo serán beneficiosas y cuándo serán excesivas. Aspira a tener un conocimiento funcional de algunos de estos algoritmos y enfoques, ya que esto te ayudará a diseñar sistemas de entrenamiento más holísticos con enfoques de ajuste de algoritmos que se complementen entre sí.

- `Hyperopt`: Hyperopt es un paquete de Python de código abierto que se presenta como adecuado para la optimización en serie y paralela sobre espacios de búsqueda complicados, que pueden incluir dimensiones de valor real, discretas y condicionales. Consulta el siguiente enlace para más información: https://github.com/Hyperopt/Hyperopt.  En el momento de escribir, la versión 0.2.5 viene empaquetada con tres algoritmos para realizar optimización sobre espacios de búsqueda proporcionados por el usuario:
    - Random search: Este algoritmo, esencialmente, selecciona números aleatorios dentro de los rangos de valores de parámetros que has proporcionado y los prueba. Luego evalúa qué conjuntos de números ofrecen el mejor rendimiento de acuerdo con la función objetivo que has elegido.
    - Tree of Parzen Estimators (TPE): Este es un enfoque de optimización bayesiana que modela distribuciones de hiperparámetros por debajo y por encima de un umbral para la función objetivo (aproximadamente buenos y malos puntajes), y luego busca extraer más valores de la distribución de hiperparámetros buenos.
    - Adaptive TPE: Esta es una versión modificada de TPE que permite cierta optimización de la búsqueda, así como la posibilidad de crear un modelo de ML para ayudar a guiar el proceso de optimización.

El repositorio y la documentación de Hyperopt contienen varios ejemplos trabajados agradables y detallados. No vamos a repasarlos aquí. En su lugar, aprenderemos cómo usar esto para un modelo de clasificación sencillo, como el que definimos en el Capítulo 1, Introducción a la Ingeniería de ML. Comencemos:

1. En Hyperopt, debemos definir los hiperparámetros que queremos optimizar. Por ejemplo, para un problema típico de regresión logística, podríamos definir el espacio de hiperparámetros a cubrir, si deseamos reutilizar los parámetros que se aprendieron de las ejecuciones anteriores del modelo cada vez (warm_start), si queremos que el modelo incluya un sesgo en la función de decisión (fit_intercept), la tolerancia establecida para decidir cuándo detener la optimización (tol), el parámetro de regularización (C), qué solver queremos probar y el número máximo de iteraciones, max_iter, en cualquier ejecución de entrenamiento:

In [ ]:
from hyperopt import hp
space = {
'warm_start' : hp.choice('warm_start', [True, False]),
'fit_intercept' : hp.choice('fit_intercept', [True, False]),
'tol' : hp.uniform('tol', 0.00001, 0.0001),
'C' : hp.uniform('C', 0.05, 2.5),
'solver' : hp.choice('solver', ['newton-cg', 'lbfgs', 'liblinear']),
'max_iter' : hp.choice('max_iter', range(10,500))
}

Luego, tenemos que definir una función objetivo para optimizar. En el caso de nuestro algoritmo de clasificación, podemos simplemente definir la función de pérdida que queremos minimizar como el f1-score. Ten en cuenta que Hyperopt permite que tu función objetivo proporcione estadísticas de ejecución y metadatos a través de tu declaración de retorno si estás utilizando la funcionalidad fmin. El único requisito si haces esto es que devuelvas un valor etiquetado como pérdida y un valor de estado válido de la lista de Hyperopt.STATUS_STRING (ok por defecto y fail si hay un problema en el cálculo que deseas señalar como un fallo):

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from functools import partial

def objective(params, n_folds, X, y):
    clf = LogisticRegression(**params, random_state=42)
    scores = cross_val_score(clf, X, y, cv=n_folds, scoring='f1_macro')
    max_score = max(scores)
    loss = 1 - max_score
    return {'loss': loss, 'params': params, 'status': STATUS_OK}

Ahora, debemos optimizar usando el método fmin con el algoritmo TPE:

In [ ]:

trials = Trials()
n_folds = 5
# Optimize
best = fmin(fn=partial(objective, n_folds=n_folds, 
                       X=X_train, y=y_train),space=space,
                       algo=tpe.suggest,max_evals=16,trials=trials
)

El mejor es un diccionario que contiene todos los mejores hiperparámetros en el espacio de búsqueda que definiste. Así que, en este caso, tenemos lo siguiente:

In [ ]:
{'C': 0.26895003542493234,
'fit_intercept': 1,
'max_iter': 452,
'solver': 2,
'tol': 1.863336145787027e-05,
'warm_start': 1}

Luego puedes usar estos hiperparámetros para definir tu modelo para entrenarlo con los datos.

- `Optuna`: Optuna es un paquete de software que tiene una amplia serie de capacidades basadas en algunos principios de diseño fundamentales, como su API definida por ejecución y su arquitectura modular. Definido por ejecución se refiere al hecho de que, al usar Optuna, el usuario no tiene que definir el conjunto completo de parámetros a probar, lo que sería definido y ejecutado. En su lugar, pueden proporcionar algunos valores iniciales y pedirle a Optuna que sugiera su propio conjunto de experimentos para ejecutar. Esto ahorra tiempo al usuario y reduce la huella de código (¡dos grandes ventajas para mí!).

Optuna contiene cuatro algoritmos de búsqueda básicos: búsqueda en cuadrícula, búsqueda aleatoria, TPE y el algoritmo de Estrategia de Evolución de Adaptación de Matriz de Covarianza (CMA-ES). Ya cubrimos los tres primeros anteriormente, pero CMA-ES es una adición importante al conjunto. Como su nombre sugiere, se basa en un algoritmo evolutivo y toma muestras de hiperparámetros de una distribución gaussiana multivariada. Luego, utiliza las clasificaciones de los puntajes evaluados para la función objetivo dada para actualizar dinámicamente los parámetros de la distribución gaussiana (la matriz de covarianza siendo un conjunto de estos) para ayudar a encontrar un óptimo en el espacio de búsqueda de manera rápida y robusta.

Sin embargo, la clave que hace que el proceso de optimización de Optuna sea diferente al de Hyperopt es su aplicación de la poda o detención temprana automatizada. Durante la optimización, si Optuna detecta evidencia de que un ensayo de un conjunto de hiperparámetros no conducirá a un algoritmo entrenado en general mejor, termina ese ensayo. Los desarrolladores del paquete sugieren que esto lleva a ganancias de eficiencia en el proceso de optimización de hiperparámetros al reducir la computación innecesaria.

Aquí, estamos viendo el mismo ejemplo que vimos anteriormente, pero ahora estamos usando Optuna en lugar de Hyperopt:

In [ ]:
import optuna

def objective(trial, n_folds, X, y):
    params = {
        'warm_start': trial.suggest_categorical('warm_start', [True, False]),
        'fit_intercept': trial.suggest_categorical('fit_intercept', [True, False]),
        'tol': trial.suggest_float('tol', 1e-5, 1e-4),   # antes suggest_uniform
        'C': trial.suggest_float('C', 0.05, 2.5),       # antes suggest_uniform
        'solver': trial.suggest_categorical('solver', ['newton-cg', 'lbfgs', 'liblinear']),
        'max_iter': trial.suggest_int('max_iter', 10, 500)  # mejor que categorical(range)
    }

    # Modelo con hiperparámetros sugeridos
    clf = LogisticRegression(**params, random_state=42)
    
    # Cross validation
    scores = cross_val_score(clf, X, y, cv=n_folds, scoring='f1_macro')
    
    # Pérdida a minimizar
    loss = 1 - max(scores)
    
    return loss


Ahora, debemos configurar los datos de la misma manera que lo hicimos en el ejemplo de Hyperopt:

In [ ]:
from sklearn import datasets
n_folds = 5
X, y = datasets.make_classification(n_samples=100000, n_features=20,n_informative=2,
n_redundant=2)
train_samples = 100
# Samples used for training the models
X_train = X[:train_samples]
X_test = X[train_samples:]
y_train = y[:train_samples]
y_test = y[train_samples:]

Ahora, podemos definir este objeto de Estudio que mencionamos y decirle cómo deseamos optimizar el valor que devuelve nuestra función objetivo, con instrucciones sobre cuántas pruebas realizar en el estudio. Aquí, utilizaremos nuevamente el algoritmo de muestreo TPE:

In [ ]:
from optuna.samplers import TPESampler
study = optuna.create_study(direction='minimize', sampler=TPESampler())
study.optimize(partial(objective, n_folds=n_folds, X=X_train, y=y_train), n_trials=16)

Ahora, podemos acceder a los mejores parámetros a través de la variable study.best_trial.params, que nos da los siguientes valores para el mejor caso:

In [ ]:
print(study.best_trial.params)

`NOTA IMPORTANTE`: Notarás que estos valores son diferentes de los devueltos por Hyperopt. Esto se debe a que solo hemos realizado 16 ensayos en cada caso, por lo que no estamos muestreando efectivamente el espacio. Si ejecutas cualquiera de las muestras de Hyperopt o Optuna varias veces seguidas, puedes obtener resultados bastante diferentes por la misma razón. El ejemplo dado aquí es solo para mostrar la sintaxis, pero si estás interesado, puedes establecer el número de iteraciones en un valor muy alto (o crear espacios más pequeños para muestrear), y los resultados de los dos enfoques deberían converger.

- `AutoML`: El nivel final de nuestra jerarquía es aquel donde nosotros, como ingenieros, tenemos el menor control directo sobre el proceso de entrenamiento, ¡pero donde también podríamos obtener una buena respuesta con muy poco esfuerzo! El tiempo de desarrollo que se requiere para buscar entre muchos hiperparámetros y algoritmos para tu problema puede ser grande, incluso cuando programas parámetros de búsqueda y bucles que parecen razonables.

Dado esto, en los últimos años se han despliegueado varias bibliotecas y herramientas de AutoML en una variedad de lenguajes y ecosistemas de software. La exageración en torno a estas técnicas ha significado que han tenido una gran cantidad de atención, lo que ha llevado a varios científicos de datos a cuestionar cuándo sus trabajos serán automatizados. Como mencionamos anteriormente en este capítulo, en mi opinión, declarar la muerte de la ciencia de datos es extremadamente prematuro y también peligroso desde el punto de vista organizacional y del rendimiento empresarial. Estas herramientas han sido otorgadas un estatus pseudo-mitológico tal que muchas empresas podrían creer que simplemente usarlas unas pocas veces resolverá todos sus problemas de ciencia de datos y ML.

Están equivocados, pero también tienen razón.

Estas herramientas y técnicas son muy poderosas y pueden ayudar a mejorar algunas cosas, pero no son una panacea mágica que se pueda usar de manera inmediata. Exploremos estas herramientas y comencemos a pensar en cómo incorporarlas en nuestro flujo de trabajo y soluciones de ingeniería de ML.


# Auto-sklearn

Una de nuestras bibliotecas favoritas, el buen viejo scikit-learn, siempre iba a ser uno de los primeros objetivos para construir una biblioteca de AutoML popular. Una de las características muy poderosas de auto-sklearn es que su API ha sido diseñada para que los objetos principales que optimizan y seleccionan modelos y hiperparámetros puedan ser intercambiados sin problemas en tu código.

Como de costumbre, un ejemplo lo mostrará con mayor claridad. En el siguiente ejemplo, asumiremos que el conjunto de datos de vino (un favorito para este capítulo) ya ha sido recuperado y dividido en muestras de entrenamiento y prueba, de acuerdo con otros ejemplos, como el que se presenta en la sección de Detección de deriva:

1. Primero, dado que este es un problema de clasificación, lo principal que necesitamos obtener de auto-sklearn es el objeto autosklearn.classification:

In [ ]:
import numpy as np
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from flaml import AutoML

# Datos
X, y = load_wine(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Configuración de AutoML
automl = AutoML()

settings = {
    "time_budget": 60,  # tiempo total en segundos
    "task": "classification",
    "metric": "accuracy",
    "log_file_name": "wine.log",
}

# Entrenamiento
automl.fit(X_train, y_train, **settings)

# Modelos y estadísticas
print("Mejor modelo:", automl.model)
print("Mejor config:", automl.best_config)
print("Mejor score (validación):", automl.best_loss)

# Predicción
predictions = automl.predict(X_test)
print("Accuracy en test:", accuracy_score(y_test, predictions))


Como puedes ver, es muy sencillo comenzar a usar esta poderosa biblioteca, especialmente si ya te sientes cómodo trabajando con sklearn.

A continuación, discutamos cómo extendemos este concepto a las redes neuronales, que tienen una capa extra de complejidad debido a sus diferentes arquitecturas de modelo potenciales.

## Optuna en redes neuronales

Un área particular donde AutoML ha tenido un gran impacto es en las redes neuronales. Esto se debe a que, para una red neuronal, la pregunta de cuál es el mejor modelo es una cuestión muy complicada. Para nuestros clasificadores típicos, generalmente podemos pensar en una lista relativamente corta y finita de algoritmos a probar. Para una red neuronal, no tenemos esta lista finita. En cambio, tenemos un conjunto esencialmente infinito de posibles arquitecturas de redes neuronales; por ejemplo, para organizar las neuronas en capas y las conexiones entre ellas. Buscar la arquitectura óptima de una red neuronal es un problema en el que una optimización poderosa puede facilitarte la vida, como ingeniero de ML o científico de datos.

En este caso, vamos a explorar a optuna

In [ ]:
import optuna
import tensorflow as tf
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow import keras

# === 1. Dataset ===
X, y = load_wine(return_X_y=True)
X = StandardScaler().fit_transform(X)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# === 2. Definir función objetivo ===
def objective(trial):
    # Hiperparámetros a explorar
    n_layers = trial.suggest_int("n_layers", 1, 3)
    units = trial.suggest_int("units", 16, 128, step=16)
    dropout = trial.suggest_float("dropout", 0.0, 0.5)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)

    # Construcción del modelo
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=(X_train.shape[1],)))
    for _ in range(n_layers):
        model.add(keras.layers.Dense(units, activation="relu"))
        model.add(keras.layers.Dropout(dropout))
    model.add(keras.layers.Dense(len(set(y)), activation="softmax"))

    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    # Entrenamiento rápido
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=30,
        batch_size=32,
        verbose=0
    )

    # Mejor accuracy en validación
    val_acc = max(history.history["val_accuracy"])
    return val_acc

# === 3. Ejecutar optimización ===
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Mejores hiperparámetros:", study.best_trial.params)

In [ ]:
print("Mejor accuracy:", study.best_trial.value)

# Persisting your models

En el capítulo anterior, presentamos algunos de los conceptos básicos del control de versiones de modelos utilizando MLflow. En particular, discutimos cómo registrar métricas para tus experimentos de ML utilizando la API de seguimiento de MLflow. Ahora vamos a construir sobre este conocimiento y considerar los puntos de contacto que nuestros sistemas de entrenamiento deben tener con los sistemas de control de modelos en general.

Primero, hagamos un resumen de lo que estamos tratando de hacer con el sistema de entrenamiento. Queremos automatizar (en la medida de lo posible) gran parte del trabajo que realizaron los científicos de datos al encontrar el primer modelo funcional, para que podamos actualizar y crear continuamente nuevas versiones del modelo que aún resuelvan el problema en el futuro. También nos gustaría tener un mecanismo simple que permita compartir los resultados del proceso de entrenamiento con la parte de la solución que llevará a cabo la predicción cuando esté en producción. Podemos pensar en nuestro sistema de control de versiones del modelo como un puente entre las diferentes etapas del proceso de desarrollo de ML que discutimos en el Capítulo 2, El Proceso de Desarrollo de Aprendizaje Automático.

En particular, podemos ver que la capacidad de rastrear los resultados de los experimentos nos permite mantener los resultados de la fase de Prueba y construir sobre ellos durante la fase de Desarrollo. También podemos rastrear más experimentos, ejecuciones de pruebas y resultados de optimización de hiperparámetros en el mismo lugar durante la fase de Desarrollo. Luego, podemos comenzar a etiquetar los modelos eficientes como aquellos que son buenos candidatos para el despliegue, cerrando así la brecha entre las fases de desarrollo de Desarrollo y Despliegue. Si nos enfocamos en MLflow por ahora (aunque hay muchas otras soluciones disponibles que cumplen con la necesidad de un sistema de control de versiones de modelos), las funcionalidades de Seguimiento y Registro de Modelos de MLflow se integran muy bien en estos roles de puente. Esto se representa esquemáticamente en el siguiente diagrama:

![Ml-flow](figures/ML-flow.png)

En el Capítulo 2, El Proceso de Desarrollo de Aprendizaje Automático, solo exploramos lo básico de la API de Seguimiento de MLflow para almacenar los metadatos de ejecución de modelos experimentales. Ahora, haremos una breve inmersión en cómo almacenar modelos listos para producción de manera muy organizada para que puedas comenzar a realizar la preparación de modelos. Este es el proceso mediante el cual los modelos pueden avanzar a través de etapas de preparación, y puedes intercambiar modelos en producción si lo deseas. Esta es una parte extremadamente importante de cualquier sistema de entrenamiento que proporciona modelos y que funcionará como parte de una solución implementada, ¡que es de lo que trata este libro!

Como se mencionó anteriormente, la funcionalidad que necesitamos en MLflow se llama Registro de Modelos, una de las cuatro funcionalidades principales de MLflow. Aquí, pasaremos por ejemplos de cómo tomar un modelo registrado y enviarlo al registro, cómo actualizar información como el número de versión del modelo en el registro, y luego cómo avanzar su modelo a través de diferentes etapas del ciclo de vida. Terminaremos esta sección aprendiendo cómo recuperar un modelo dado del registro en otros programas, un punto clave si queremos compartir nuestros modelos entre servicios de entrenamiento y predicción separados.

Antes de sumergirnos en el código de Python para interactuar con el Registro de Modelos, tenemos una pieza importante de configuración que realizar. El registro solo funciona si se está utilizando una base de datos para almacenar los metadatos y parámetros del modelo. Esto es diferente de la API de Seguimiento básica, que funciona solo con un almacén de archivos. Esto significa que antes de enviar modelos al Registro de Modelos, debemos iniciar un servidor de MLflow con un backend de base de datos. Puedes hacer esto con una base de datos PostgreSQL ejecutándose localmente al ejecutar el siguiente comando en tu terminal. Tendrás que ejecutar esto antes de los fragmentos de código en el resto de esta sección.




Ahora que la base de datos del backend está en funcionamiento, podemos usarla como parte de nuestro flujo de trabajo del modelo. Empecemos:

1. Comencemos registrando algunas métricas y parámetros para uno de los modelos que entrenamos anteriormente en este capítulo:

input_example = pd.DataFrame(X_train[:1], columns=wine.feature_names)

¿Por qué se hace esto?

Ese input_example se pasa al registrar el modelo en MLflow.

MLflow lo usa para inferir la firma del modelo automáticamente:

- número de columnas,

- nombres de features,

- tipos de datos (float, int, etc.).

Con eso, cuando vayas a servir el modelo (por ejemplo, vía API), MLflow sabrá qué tipo de datos espera y cómo validarlos

In [ ]:
import mlflow
import mlflow.sklearn
import pandas as pd

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeClassifier
from sklearn import metrics
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split


# Conectar el script al servidor MLflow
mlflow.set_tracking_uri("http://0.0.0.0:5000")
# === 1. Dataset ===
wine = load_wine()
X, y = load_wine(return_X_y=True)
X = pd.DataFrame(wine.data, columns=wine.feature_names) 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear un input_example (para que MLflow infiera la firma)
input_example = pd.DataFrame(X_train[:1], columns=wine.feature_names)

with mlflow.start_run(run_name="YOUR_RUN_NAME") as run:
    params = {'tol': 1e-2,'solver': 'sag'}
    std_scale_clf = make_pipeline(StandardScaler(), RidgeClassifier(**params))

    std_scale_clf.fit(X_train, y_train)
    y_pred_std_scale = std_scale_clf.predict(X_test)

    mlflow.log_metrics(
        {
            'accuracy': metrics.accuracy_score(y_test, y_pred_std_scale),
            'precision': metrics.precision_score(y_test, y_pred_std_scale,average='macro'),
            'f1': metrics.f1_score(y_test, y_pred_std_scale, average='macro'),
            'recall': metrics.recall_score(y_test, y_pred_std_scale,average='macro')
        }
    )

    mlflow.log_params(params)

    """Dentro del mismo bloque de código, ahora podemos registrar el modelo en el Registro de Modelos, 
    proporcionando un nombre para hacer referencia al modelo más tarde:
    """
    # Registrar el modelo en el Model Registry con input_example
    mlflow.sklearn.log_model(
        sk_model=std_scale_clf,
        name="sk-learn-std-scale-clf",   # reemplaza artifact_path
        registered_model_name="sk-learn-std-scale-clf",
        input_example=input_example
    )

Ahora, supongamos que estamos ejecutando un servicio de predicción y queremos recuperar el modelo y predecir utilizando este. Aquí, tenemos que escribir lo siguiente:

In [ ]:
model_name = "sk-learn-std-scale-clf"
model_version = 3
model = mlflow.pyfunc.load_model(
    model_uri=f"models:/{model_name}/{model_version}"
)
X_test = pd.DataFrame(X_test, columns=wine.feature_names)
model.predict(X_test)

Por defecto, los modelos recién registrados en el Registro de Modelos se asignan al valor de etapa 'None'. Por lo tanto, debemos agregarle un alias de la siguiente forma:

In [ ]:
from mlflow.tracking import MlflowClient
client = MlflowClient()
client.set_registered_model_alias(
    name="sk-learn-std-scale-clf",
    alias="staging",
    version=3
)

Basado en todas nuestras conversaciones en este capítulo, el resultado de nuestro sistema de entrenamiento debe ser capaz de producir un modelo que estemos satisfechos de implementar en producción. La siguiente pieza de código promociona el modelo a una etapa diferente, llamada "Producción":.

In [ ]:
# Asignar el alias "production" al modelo versión 1
client.set_registered_model_alias(
    name="sk-learn-std-scale-clf",
    alias="production",
    version=3
)

Estas son las formas más importantes de interactuar con el Registro de Modelos y hemos cubierto lo básico sobre cómo registrar, actualizar, promover y recuperar tus modelos en tus sistemas de entrenamiento (y predicción).

Ahora, aprenderemos cómo encadenar nuestros principales pasos de entrenamiento en unidades singulares llamadas pipelines. Cubriremos algunas de las formas estándar de hacer esto dentro de scripts únicos, lo que nos permitirá construir nuestros primeros pipelines de entrenamiento. En el Capítulo 5, Patrones y Herramientas de Despliegue, cubriremos herramientas para construir pipelines de software más genéricos para tu solución de ML (de los cuales tu pipeline de entrenamiento puede ser un único componente).

# Construyendo la fabrica de modelos con pipelines

El concepto de un pipeline de software es lo suficientemente intuitivo. Si tienes una serie de pasos encadenados en tu código, de manera que el siguiente paso consume o utiliza la salida del paso o pasos anteriores, entonces tienes un pipeline.


## Scikit-learn pipelines

Nuestro viejo amigo scikit-learn viene empaquetado con una agradable funcionalidad de creación de pipelines. En el momento de escribir esto, las versiones de scikit-learn superiores a 0.20 también contienen el objeto ColumnTransformer, que te permite construir pipelines que realizan diferentes acciones en columnas específicas. Esto es exactamente lo que queremos hacer con el ejemplo del modelo de marketing de regresión logística que discutíamos anteriormente, donde queremos estandarizar nuestros valores numéricos y codificar en dummys nuestras variables categóricas. Comencemos:

1. Para crear esta pipeline, necesitas importar los objetos ColumnTransformer y Pipeline:

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import os
import pandas as pd

file_path = lambda file: os.path.join(os.getcwd(),'data',file)
bank_df = pd.read_csv(file_path('bank.csv'))

2. Ahora, debemos definir el sub-pipeline del transformador numérico, que contiene los dos pasos para la imputación y la escalación. También debemos definir los nombres de las columnas numéricas a las que se aplicará esto para que podamos usarlas más tarde:

In [ ]:
numeric_features = ['age', 'balance']
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
    ])

3. A continuación, debemos realizar pasos similares para las variables categóricas, pero aquí, solo tenemos un paso de transformación que definir para el codificador one-hot:

In [ ]:
from sklearn.preprocessing import OneHotEncoder
categorical_features = ['job', 'marital', 'education', 'contact', 'housing', 'loan','default','day']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

5. Debemos reunir todos estos pasos de preprocesamiento en un solo objeto, llamado preprocesador, utilizando el objeto ColumnTransformer. Esto aplicará nuestros transformadores a las columnas apropiadas de nuestro DataFrame:

In [ ]:
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_features),
    ('cat', categorical_transformer, categorical_features)
    ])

6. Finalmente, queremos agregar el paso del modelo de ML al final de los pasos anteriores y finalizar la pipéline. Llamaremos a esto clf_pipeline:

In [ ]:
from sklearn.linear_model import LogisticRegression
clf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())
    ])

Este es nuestro primer pipeline de entrenamiento de ML. La belleza de la API de scikit-learn es que el objeto clf_pipeline ahora puede ser llamado como si fuera un algoritmo estándar del resto de la biblioteca. Así que esto significa que podemos escribir lo siguiente:

In [ ]:
X = bank_df.drop(columns=['deposit'],axis=1)
y = bank_df['deposit']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
clf_pipeline.fit(X_train, y_train)

Esto ejecutará los métodos de ajuste de todos los pasos del pipeline en secuencia. La capacidad de abstraer los pasos que realizan la ingeniería de características y entrenan tu modelo en un solo objeto es muy poderosa, ya que significa que puedes exportar e importar este pipeline en varios lugares, sin conocer los detalles de la implementación. ¡La abstracción es algo bueno!

## Spark ML pipelines

Hay otro conjunto de herramientas que hemos estado utilizando a lo largo de este libro que será particularmente importante cuando discutamos cómo escalar nuestras soluciones: Apache Spark y su ecosistema de ML. Veremos que construir un pipeline similar con Spark ML requiere un conjunto de sintaxis ligeramente diferente, pero los conceptos clave son muy similares al caso de sklearn.

Hay algunos puntos importantes que mencionar sobre los pipelines de PySpark. En primer lugar, de acuerdo con las buenas prácticas de programación en Scala, en el que está escrito Spark, los objetos se tratan como inmutables, por lo que las transformaciones no ocurren en su lugar. En su lugar, se crean nuevos objetos. Esto significa que la salida de cualquier transformación requerirá que se creen nuevas columnas en tu DataFrame original (o de hecho nuevas columnas en un nuevo DataFrame).

En segundo lugar, los estimadores de Spark MLlib (es decir, los algoritmos de ML) requieren que las características se ensamblen en un único objeto similar a una tupla en una sola columna. Esto contrasta con scikit-learn, donde puedes mantener todas las características en sus columnas en tu objeto de datos. Esto significa que necesitas acostumbrarte al uso de ensambladores, utilidades para unir columnas de características dispares, especialmente cuando trabajas con características categóricas y numéricas mixtas que deben transformarse de diferentes maneras antes de ser utilizadas por el algoritmo.

En tercer lugar, Spark tiene muchas funciones que utilizan evaluación perezosa, lo que significa que solo se ejecutan cuando son activadas por acciones específicas. Esto significa que puedes construir todo tu pipeline de ML sin tener que transformar ningún dato. La razón de la evaluación perezosa es que los pasos computacionales en Spark se almacenan en un Grafo Acíclico Dirigido (DAG) para que el plan de ejecución pueda ser optimizado antes de que realices los pasos computacionales, lo que hace que Spark sea muy eficiente.

Finalmente – y este es un punto menor – es común escribir variables de PySpark usando camel case en lugar del común snake case, que a menudo se usa para las variables de Python (por ejemplo, variableName frente a variable_name). Esto se hace para mantener el código en línea con las funciones de PySpark que heredan esta convención del código Scala subyacente detrás de Spark. Echemos un vistazo:

1. Primero, debemos crear la sección de spark
2. segundo, debemos leer los datos

In [1]:
#1
from pyspark import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext('local')
spark = SparkSession(sc)

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/12 10:34:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [10]:
#2
import os
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType
from pyspark.sql.functions import when

file_path = lambda file: os.path.join(os.getcwd(),'data',file)
schema = StructType([
    StructField("age", IntegerType(), True),
    StructField("job", StringType(), True),
    StructField("marital", StringType(), True),
    StructField("education", StringType(), True),
    StructField("default", StringType(), True),
    StructField("balance", DoubleType(), True),
    StructField("housing", StringType(), True),
    StructField("loan", StringType(), True),
    StructField("contact", StringType(), True),
    StructField("day", StringType(), True),
    StructField("month", StringType(), True),
    StructField("duration", StringType(), True),
    StructField("campaign", StringType(), True),
    StructField("pdays", StringType(), True),
    StructField("previous", StringType(), True),
    StructField("poutcome", StringType(), True),
    StructField("deposit", StringType(), True)
])

df = spark.read.csv(file_path('bank.csv'), sep=',', header=True, schema=schema)
# Transformar la columna 'deposit' de 'yes'/'no' a 1/0
df = df.withColumn("deposit", when(df["deposit"] == "yes", 1).otherwise(0))
df.show(10)

+---+----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
|age|       job| marital|education|default|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|deposit|
+---+----------+--------+---------+-------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+-------+
| 59|    admin.| married|secondary|     no| 2343.0|    yes|  no|unknown|  5|  may|    1042|       1|   -1|       0| unknown|      1|
| 56|    admin.| married|secondary|     no|   45.0|     no|  no|unknown|  5|  may|    1467|       1|   -1|       0| unknown|      1|
| 41|technician| married|secondary|     no| 1270.0|    yes|  no|unknown|  5|  may|    1389|       1|   -1|       0| unknown|      1|
| 55|  services| married|secondary|     no| 2476.0|    yes|  no|unknown|  5|  may|     579|       1|   -1|       0| unknown|      1|
| 54|    admin.| married| tertiary|     no|  184.0|     no|  no|unkno

In [11]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: double (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: string (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- campaign: string (nullable = true)
 |-- pdays: string (nullable = true)
 |-- previous: string (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: integer (nullable = false)



In [12]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, Imputer, VectorAssembler, StandardScaler

# se definen las columnas numéricas y string
categoricalColumns = ["job", "marital", "education", "contact", "housing", "loan", "default", "day"]
numericalColumns = ["age", "balance"]

In [13]:
# Crear etapas para columnas categóricas
indexers = [StringIndexer(inputCol=col, outputCol=col + "Index").setHandleInvalid("keep") for col in categoricalColumns]
encoders = [OneHotEncoder(inputCols=[indexer.getOutputCol()], outputCols=[col + "classVec"]) for indexer, col in zip(indexers, categoricalColumns)]

In [14]:
# Imputar valores nulos en columnas numéricas
imputer = Imputer(inputCols=numericalColumns, 
                  outputCols=[col + "_imputed" for col in numericalColumns])

In [15]:
# Ensamblar columnas numéricas imputadas
numericalAssembler = VectorAssembler(inputCols=[col + "_imputed" for col in numericalColumns], 
                                     outputCol='numerical_cols_imputed')

In [16]:
# Escalar las columnas numéricas
scaler = StandardScaler(inputCol='numerical_cols_imputed', outputCol="numerical_cols_imputed_scaled")

# Ensamblar todas las características
assemblerInputs = [col + "classVec" for col in categoricalColumns] + ["numerical_cols_imputed_scaled"]
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")

# Crear la lista de etapas
stages = indexers + encoders + [imputer, numericalAssembler, scaler, assembler]

Finalmente, podemos definir nuestro paso de modelo, añadir esto al pipeline y luego entrenar y transformar los datos apropiados:

In [17]:
from pyspark.ml.classification import LogisticRegression

# Definir el modelo de regresión logística
lr = LogisticRegression(labelCol="deposit", featuresCol="features", maxIter=10)

# Agregar el modelo a las etapas
stages += [lr]

# Crear el pipeline
clfPipeline = Pipeline(stages=stages)

# Dividir los datos en conjuntos de entrenamiento y prueba
(trainingData, testData) = df.randomSplit([0.7, 0.3], seed=100)

# Ajustar el pipeline
model = clfPipeline.fit(trainingData)

# Transformar los datos de prueba
predictions = model.transform(testData)

25/09/12 10:39:24 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Y así es como podemos construir un pipeline de entrenamiento en PySpark.

# Resumen

En este capítulo, aprendimos sobre el importante tema de cómo construir nuestras soluciones para entrenar y desplegar los modelos de ML que queremos ejecutar en producción. Dividimos los componentes de dicha solución en partes que abordaban el entrenamiento de los modelos, la persistencia de los modelos, el servicio de los modelos y la activación del reentrenamiento de los modelos.

Realizamos una investigación detallada sobre las razones por las cuales podrías querer separar tus componentes de entrenamiento y ejecución por razones de rendimiento. Luego discutimos cómo puedes realizar detección de deriva en el rendimiento de tu modelo y en las estadísticas de datos para comprender si se debe activar un nuevo entrenamiento. A continuación, resumimos algunos de los conceptos clave de la ingeniería de características, o cómo transformas tus datos en algo que un modelo de ML puede entender. Luego profundizamos en cómo aprenden los modelos de ML y qué aspectos de ese proceso puedes controlar, y terminamos esa sección revisando algunas herramientas que permiten ejercer ese control. Luego cubrimos cómo persistir y recuperar modelos utilizando simples comandos de Python que se pueden ejecutar desde tus sistemas de entrenamiento y predicción. Finalmente, abordamos el concepto de pipelines de entrenamiento y cómo estos objetos simplifican tu solución de entrenamiento.

En el próximo capítulo, descubriremos cómo empaquetar algunos de estos conceptos de una manera pythonica para que puedan ser implementados y reutilizados sin problemas en otros proyectos.
